## AC Transit GTFS - Bus Bunching (IN PROGRESS)

This notebook loads in route, stop, and trip information and locations collected from the AC Transit API, then uses geospatial analysis to identify bus bunching.

**Links**: 
- [AC Transit API Documentation](https://api.actransit.org/transit/Help)
- [Bus Bunching Explained Visually](https://setosa.io/bus/)

In [2]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime as dt

In [3]:
import folium
import matplotlib.pyplot as plt
import geopandas as gpd

from geopy.distance import great_circle

In [4]:
# import stops from .CSV file
stop_patterns_df = pd.read_csv("../data/ACTransit_StopPatterns.csv")

# convert dataframe to geodataframe
stop_patterns_gdf = gpd.GeoDataFrame(stop_patterns_df, 
                                     geometry=gpd.points_from_xy(stop_patterns_df.lon, stop_patterns_df.lat))

In [5]:
stop_patterns_gdf.head()

,RouteId,rtdir,lat,lon,stpid,stpnm,geometry
0,6,To Downtown Berkeley,37.802254,-122.274456,51636,10th St:Washington St,POINT (-122.27446 37.80225)
1,6,To Downtown Berkeley,37.802963,-122.274464,59444,11th St + Clay St,POINT (-122.27446 37.80296)
2,6,To Downtown Berkeley,37.804103,-122.269714,53602,Franklin St:14th St,POINT (-122.26971 37.80410)
3,6,To Downtown Berkeley,37.809587,-122.269413,57999,Thomas L Berkley Way + Telegraph Av,POINT (-122.26941 37.80959)
4,6,To Downtown Berkeley,37.814091,-122.268358,59991,Telegraph Av + 24th St,POINT (-122.26836 37.81409)


In [6]:
# import trips from .CSV file
trips_df = pd.read_csv("../data/ACTransit_Trips.csv")

trips_df.head()

,RouteId,Direction,TripId,StartTime
0,1,Northbound,7279073,2000-01-01T00:49:00
1,1,Northbound,7279074,2000-01-01T01:49:00
2,1,Northbound,7279075,2000-01-01T02:49:00
3,1,Northbound,7279076,2000-01-01T03:49:00
4,1,Northbound,7279077,2000-01-01T04:49:00


In [27]:
# read in GTFS RT data
vehloc_df = pd.read_csv("../data/ACTransit_Veh_Loc_0303.csv")

# convert time column to a datetime
vehloc_df.time = pd.to_datetime(vehloc_df.time)

# convert trip_id to integer and drop extra values
vehloc_df.trip_id = vehloc_df.trip_id[0].astype(str)[:-4]
vehloc_df.trip_id = vehloc_df.trip_id.astype(int)

# convert dataframe to geodataframe
vehloc_gdf = gpd.GeoDataFrame(vehloc_df, geometry=gpd.points_from_xy(vehloc_df.longitude, vehloc_df.latitude))

In [28]:
vehloc_gdf.head()

,route_id,trip_id,vehicle_id,latitude,longitude,bearing,speed,unix_time,time,geometry
0,19,7422200,5020,37.767731,-122.233131,167.0,0.00000,1583269548,2020-03-03 13:05:48-08:00,POINT (-122.23313 37.76773)
1,34,7422200,5022,37.686657,-122.138054,88.0,10.72896,1583269549,2020-03-03 13:05:49-08:00,POINT (-122.13805 37.68666)
2,34,7422200,5023,37.718044,-122.178970,274.0,0.00000,1583269544,2020-03-03 13:05:44-08:00,POINT (-122.17897 37.71804)
3,212,7422200,5025,37.526546,-122.005966,304.0,0.00000,1583269546,2020-03-03 13:05:46-08:00,POINT (-122.00597 37.52655)
4,51A,7422200,1341,37.775002,-122.224571,305.0,0.00000,1583269541,2020-03-03 13:05:41-08:00,POINT (-122.22457 37.77500)


In [31]:
trips_temp = trips_df[trips_df.RouteId =='51B'].TripId.unique()
vehloc_gdf[vehloc_gdf.trip_id.isin(trips_temp)]

,route_id,trip_id,vehicle_id,latitude,longitude,bearing,speed,unix_time,time,geometry


In [130]:
# https://stackoverflow.com/questions/56520780/how-to-use-geopanda-or-shapely-to-find-nearest-point-in-same-geodataframe

# NOT WHAT I WANT

from shapely.geometry import Point
from shapely.ops import nearest_points

for index, row in vehloc_gdf.iterrows():
    point = row.geometry
    multipoint = stop_patterns_gdf.drop(index, axis=0).geometry.unary_union
    queried_geom, nearest_geom = nearest_points(point, multipoint)
    #print(queried_geom, nearest_geom)
    #vehloc_gdf.loc[index, 'nearest_geometry'] = nearest_geom

POINT (-122.2331314086914 37.76773071289063) POINT (-122.2337699999999 37.7685809924771)


TypeError: float() argument must be a string or a number, not 'Point'

In [100]:
test_df = vehloc_df[(vehloc_df.route_id == "51B") & (vehloc_df.speed > 0)].copy().reset_index(drop=True)
locations = test_df[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [115]:
test_df.head()

,route_id,trip_id,vehicle_id,latitude,longitude,bearing,speed,unix_time,time,hour,minute
0,51B,741888020.0,1575,37.852180,-122.252563,175.0,8.94080,1583269542,2020-03-03 13:05:42-08:00,13,5
1,51B,741931020.0,1551,37.869164,-122.292000,259.0,5.81152,1583269551,2020-03-03 13:05:51-08:00,13,5
2,51B,741909020.0,1583,37.850929,-122.252563,164.0,3.12928,1583269609,2020-03-03 13:06:49-08:00,13,6
3,51B,742059020.0,1205,37.868687,-122.266151,260.0,7.15264,1583269655,2020-03-03 13:07:35-08:00,13,7
4,51B,741998020.0,1348,37.868889,-122.257790,259.0,0.89408,1583269732,2020-03-03 13:08:52-08:00,13,8


In [119]:
test_df["hr_min"] = test_df.time.dt.strftime("%H:%M")
#test_df["minute"] = test_df.time.dt.minute

In [121]:
# for t in test_df.hr_min.unique():
#     temp_df = test_df[test_df.hr_min==t].copy()
#     for row in temp_df:
          #https://stackoverflow.com/questions/37149358/apply-function-to-pandas-dataframe-row-using-values-in-other-rows
#         df['FORECAST'] = df.apply(lambda x: 

array(['13:05', '13:06', '13:07', '13:08', '13:09', '13:10', '13:11',
       '13:12', '13:13', '13:14', '13:15', '13:16', '13:18', '13:17',
       '13:19', '13:20', '13:21', '13:23', '13:22', '13:24', '13:26',
       '13:25', '13:27', '13:28', '13:29', '13:30', '13:31', '13:32',
       '13:33', '13:34', '13:35', '13:36', '13:38', '13:39', '13:40',
       '13:41', '13:42', '13:44', '13:43', '13:45', '13:46', '13:47',
       '13:48', '13:49', '13:50', '13:51', '13:52', '13:53', '13:54',
       '13:55', '13:56', '13:57', '13:58', '13:59', '14:00', '14:01',
       '14:02', '14:03', '14:04', '14:06', '14:07', '14:08', '14:09',
       '14:10', '14:11', '14:12', '15:32', '15:33', '16:11', '16:12',
       '16:13', '16:14', '16:15', '16:16', '16:17', '16:18', '16:19',
       '16:20'], dtype=object)

In [120]:
test_df.groupby(["hour", "minute"]).head()

,route_id,trip_id,vehicle_id,latitude,longitude,bearing,speed,unix_time,time,hour,minute,hr_min
0,51B,741888020.0,1575,37.852180,-122.252563,175.0,8.94080,1583269542,2020-03-03 13:05:42-08:00,13,5,13:05
1,51B,741931020.0,1551,37.869164,-122.292000,259.0,5.81152,1583269551,2020-03-03 13:05:51-08:00,13,5,13:05
2,51B,741909020.0,1583,37.850929,-122.252563,164.0,3.12928,1583269609,2020-03-03 13:06:49-08:00,13,6,13:06
3,51B,742059020.0,1205,37.868687,-122.266151,260.0,7.15264,1583269655,2020-03-03 13:07:35-08:00,13,7,13:07
4,51B,741998020.0,1348,37.868889,-122.257790,259.0,0.89408,1583269732,2020-03-03 13:08:52-08:00,13,8,13:08
...,...,...,...,...,...,...,...,...,...,...,...,...
246,51B,741892020.0,1575,37.870476,-122.268150,167.0,7.15264,1583281156,2020-03-03 16:19:16-08:00,16,19,16:19
247,51B,741935020.0,1551,37.871181,-122.267723,355.0,4.91744,1583281153,2020-03-03 16:19:13-08:00,16,19,16:19
248,51B,741913020.0,1583,37.850327,-122.252380,175.0,4.91744,1583281228,2020-03-03 16:20:28-08:00,16,20,16:20
249,51B,742063020.0,1205,37.868416,-122.262032,264.0,7.59968,1583281218,2020-03-03 16:20:18-08:00,16,20,16:20


In [88]:
map = folium.Map(location=[37.8, -122.2], tiles='CartoDB positron', zoom_start=11)
for point in range(0, len(locationlist)):
    folium.Marker(
        locationlist[point], 
        popup=folium.Popup(
            "Route: " + test_df['route_id'][point] + "\n" + 
            "Time"
        )
    ).add_to(map)
map